In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('personal.csv')

In [9]:
df

,indice,nombre,sexo,edad,area,horario,performance
0,0,Renee Rodriguez,F,30,Calles y Aceras,Noche,601
1,1,Denise Gonzales,F,32,Calles y Aceras,Tarde,262
2,2,Richard Wagner,M,28,Areas Verdes,Dia,479
3,3,Fernando Gibson,F,49,Areas Verdes,Tarde,281
4,4,Aaron Wade,M,53,Areas Verdes,Noche,436
...,...,...,...,...,...,...,...
299995,299995,Ruben Fernandez,F,42,Oficinas,Tarde,263
299996,299996,George Sanchez,F,37,Oficinas,Tarde,321
299997,299997,Tara Shaffer,M,32,Calles y Aceras,Dia,381
299998,299998,John Ryan,M,35,Calles y Aceras,Dia,469


In [10]:
df = df.drop(['indice', 'nombre', 'sexo'], axis=1)

In [11]:
df

,edad,area,horario,performance
0,30,Calles y Aceras,Noche,601
1,32,Calles y Aceras,Tarde,262
2,28,Areas Verdes,Dia,479
3,49,Areas Verdes,Tarde,281
4,53,Areas Verdes,Noche,436
...,...,...,...,...
299995,42,Oficinas,Tarde,263
299996,37,Oficinas,Tarde,321
299997,32,Calles y Aceras,Dia,381
299998,35,Calles y Aceras,Dia,469


In [12]:
def get_range(edad):
  edades = [
      [18, 30, 1],
      [31, 45, 2],
      [46, 55, 3],
      [56, 99, 4],
  ]
  for r in edades:
    if r[0] <= edad and edad <= r[1]:
      return r[2]

df["range_edad"] = df.apply(lambda x: get_range(x["edad"]), axis=1)

In [13]:
df = df.drop(["edad"], axis=1)
df.dropna()
df

,area,horario,performance,range_edad
0,Calles y Aceras,Noche,601,1
1,Calles y Aceras,Tarde,262,2
2,Areas Verdes,Dia,479,1
3,Areas Verdes,Tarde,281,3
4,Areas Verdes,Noche,436,3
...,...,...,...,...
299995,Oficinas,Tarde,263,2
299996,Oficinas,Tarde,321,2
299997,Calles y Aceras,Dia,381,2
299998,Calles y Aceras,Dia,469,2


In [14]:
df_grouped = df.groupby(['area', 'horario', 'range_edad']).agg(['sum', 'count', 'mean', 'min', 'max'])
df_grouped

performance                             
                                           sum  count        mean  min  max
area            horario range_edad                                         
Areas Verdes    Dia     1              4448553   8907  499.444594  400  599
                        2              5085989  10721  474.395019  380  569
                        3              3055376   7195  424.652675  340  509
                        4              1897730   6341  299.279293  240  359
                Noche   1              4618487   9227  500.540479  400  599
                        2              5044202  10636  474.257428  380  569
                        3              3015906   7115  423.879972  340  509
                        4              1871842   6252  299.398912  240  359
                Tarde   1              3212846   9192  349.526327  300  399
                        2              3546699  10694  331.653170  285  379
                        3              2095651   7062  296.750354  255  339
                        4              1360311   6498  209.343029  180  239
Calles y Aceras Dia     1              4569665   9143  499.799300  400  599
                        2              5048458  10647  474.167183  380  569
                        3              2976890   7021  423.998006  340  509
                        4              1885713   6301  299.272020  240  359
                Noche   1              6485123   9282  698.677332  600  799
                        2              7057342  10625  664.220424  570  759
                        3              4246269   7156  593.385830  510  679
                        4              2602794   6216  418.724903  360  479
                Tarde   1              2283941   9163  249.256903  200  299
                        2              2514482  10639  236.345709  190  284
                        3              1496260   7082  211.276476  170  254
                        4               955825   6399  149.370995  120  179
Oficinas        Dia     1              2770622   9257  299.300205  200  399
                        2              3022770  10657  283.641738  190  379
                        3              1807942   7125  253.746246  170  339
                        4              1140574   6361  179.307342  120  239
                Noche   1              6371020   9111  699.266820  600  799
                        2              7275529  10965  663.522937  570  759
                        3              4245649   7135  595.045410  510  679
                        4              2684523   6402  419.325679  360  479
                Tarde   1              2754566   9174  300.257903  200  399
                        2              3006481  10584  284.059051  190  379
                        3              1837880   7232  254.131637  170  339
                        4              1162947   6483  179.384081  120  239

In [73]:
def get_series(df_g, mts_series, sample):
  data = []
  # print(len(df_g))
  for serie in mts_series:
    s = df_g.sample(frac=1, replace=False, random_state=1, axis=0)
    counter2 = 0
    counter = 0
    sum = 0
    sample_count = 0
    for s1 in s.itertuples():
      counter = counter + 1
      sum = sum + s1[3]
      if sum >= serie:
        # print(f"{s1[1]} - {s1[2]} - {s1[4]} - {s1[3]} - {sum} = {counter} - {serie}")
        data.append([s1[1], s1[2], s1[4], sum, counter, serie])
        sum = 0
        counter = 0
        sample_count = sample_count + 1
      
      if sample_count >= sample:
        break
      
      counter2 + counter2 + 1
      if counter2 == 3:
        break
    # break

  return data


In [53]:
mts = [
    50, 100, 150,
    200, 250, 300,
    350, 400, 450,
    500, 550, 600,
    650, 700, 800,
    900, 1000, 1100,
    1200, 1300, 1400,
    1500, 1600, 1700,
    1800, 1900, 2000,
    2500, 3000, 4000,
    5000, 7000, 8000,
    9000, 10000, 11000,
    12000, 13000, 14000,
    15000, 1600, 1700,
    1800, 1900, 2000,
    22000, 24000, 26000,
    28000, 30000, 33000,
    36000, 39000, 42000,
    45000, 50000, 55000,
    60000, 70000, 80000,
    90000, 100000, 110000,
    120000, 130000, 140000,
    150000, 160000, 170000,
    180000, 190000, 200000,
]
print(len(mts))
print(sum(mts))

72
2525950


In [76]:
df1_grouped = df.groupby(['area', 'horario', 'range_edad'])

final_data = []
for group_name, df_group in df1_grouped:
  print(f"group_name: {group_name}")
  final_data = final_data + get_series(df_group, mts, sample=990)
  # print(f"df_group: \n {df_group}")
  # cont = 1
  # for row in df_group.itertuples():
  #   print(f"row: {row}")
  #   if cont == 10:
  #     break
  #   cont += 1

# group_name: ('Areas Verdes', 'Dia', 1)
# 8907
# Pandas(Index=7701, area='Areas Verdes', horario='Dia', performance=478, range_edad=1)
# Pandas(Index=175468, area='Areas Verdes', horario='Dia', performance=445, range_edad=1)
# Pandas(Index=29063, area='Areas Verdes', horario='Dia', performance=471, range_edad=1)
# Pandas(Index=261669, area='Areas Verdes', horario='Dia', performance=484, range_edad=1)


# group_name: ('Areas Verdes', 'Dia', 1)
# 8907
# Pandas(Index=20330, area='Areas Verdes', horario='Dia', performance=533, range_edad=1)
# Pandas(Index=252670, area='Areas Verdes', horario='Dia', performance=582, range_edad=1)
# Pandas(Index=182324, area='Areas Verdes', horario='Dia', performance=438, range_edad=1)
# Pandas(Index=241424, area='Areas Verdes', horario='Dia', performance=411, range_edad=1)

group_name: ('Areas Verdes', 'Dia', 1)
group_name: ('Areas Verdes', 'Dia', 2)
group_name: ('Areas Verdes', 'Dia', 3)
group_name: ('Areas Verdes', 'Dia', 4)
group_name: ('Areas Verdes', 'Noche', 1)
group_name: ('Areas Verdes', 'Noche', 2)
group_name: ('Areas Verdes', 'Noche', 3)
group_name: ('Areas Verdes', 'Noche', 4)
group_name: ('Areas Verdes', 'Tarde', 1)
group_name: ('Areas Verdes', 'Tarde', 2)
group_name: ('Areas Verdes', 'Tarde', 3)
group_name: ('Areas Verdes', 'Tarde', 4)
group_name: ('Calles y Aceras', 'Dia', 1)
group_name: ('Calles y Aceras', 'Dia', 2)
group_name: ('Calles y Aceras', 'Dia', 3)
group_name: ('Calles y Aceras', 'Dia', 4)
group_name: ('Calles y Aceras', 'Noche', 1)
group_name: ('Calles y Aceras', 'Noche', 2)
group_name: ('Calles y Aceras', 'Noche', 3)
group_name: ('Calles y Aceras', 'Noche', 4)
group_name: ('Calles y Aceras', 'Tarde', 1)
group_name: ('Calles y Aceras', 'Tarde', 2)
group_name: ('Calles y Aceras', 'Tarde', 3)
group_name: ('Calles y Aceras', 'Tarde',

In [78]:
import numpy as np
df_final = pd.DataFrame(np.array(final_data),columns=['area','horario','rango_edad','performance_persona','cantidad_personas', 'mt2_solicitados'])
df_final

,area,horario,rango_edad,performance_persona,cantidad_personas,mt2_solicitados
0,Areas Verdes,Dia,1,533,1,50
1,Areas Verdes,Dia,1,582,1,50
2,Areas Verdes,Dia,1,438,1,50
3,Areas Verdes,Dia,1,411,1,50
4,Areas Verdes,Dia,1,414,1,50
...,...,...,...,...,...,...
1384049,Oficinas,Tarde,4,200177,1106,200000
1384050,Oficinas,Tarde,4,200098,1126,200000
1384051,Oficinas,Tarde,4,200012,1102,200000
1384052,Oficinas,Tarde,4,200056,1117,200000


In [79]:
df_final.to_csv('/content/demandas.csv')